In [ ]:
# Import dependencies
import os as os
from glob import glob

import numpy as np
from nptyping import NDArray
from skimage.metrics import structural_similarity, mean_squared_error
from skimage.io import imsave
from skimage.filters import meijering
from tifffile import imread

import pandas as pd

In [ ]:
# Extract ridges and compute SSIM for ExM and ExSRRF images
src_path = '/path_to_ExM_ExSRRF/'

src_files = [fn for fn in glob(os.path.join(src_path, '*.tif'))]

# Optionally restrict the number of images that will be evaluated
n_files_max: int = len(src_files)

out_path = '/output_path/'

ssim_exsrrf = []
ssim_ex = []
img_identifier_list = []

for src_file_path in src_files:

    img_identifier = str(src_file_path.split('/')[-1].split('.tif')[0])
    print('Evaluating image ' + img_identifier)
    img_identifier_list.append(img_identifier)

    try:
        [img_ex, img_exsrrf] = imread(src_file_path)
    except:
        print('Problem processing file ', img_identifier)
        continue
    
    ridges_ex = meijering(img_ex, black_ridges=False)
    ridges_exsrrf = meijering(img_exsrrf, black_ridges=False)
    
    ssim_ex.append(structural_similarity(img_ex, ridges_ex))
    ssim_exsrrf.append(structural_similarity(img_exsrrf, ridges_exsrrf))
    
    imsave(out_path + '/ridges_exm_' + img_identifier + '.png', 
           ridges_ex)
    imsave(out_path + '/ridges_exsrrf_' + img_identifier + '.png', 
           ridges_exsrrf)
    
# Extract ridges and compute SSIM for STED images
src_path_sted = '/path_to_STED/'

src_files = [fn for fn in glob(os.path.join(src_path_sted, '*.tif'))]

# Optionally restrict the number of images that will be evaluated
n_files_max: int = len(src_files)

ssim_sted = []
img_identifier_list_sted = []

for src_file_path in src_files[:n_files_max]:

    img_identifier = str(src_file_path.split('/')[-1].split('.tif')[0])
    print('Evaluating image ' + img_identifier)
    img_identifier_list_sted.append(img_identifier)

    try:
        img_sted = imread(src_file_path)
    except:
        print('Problem processing file ', img_identifier)
        continue

    # Extract ridges
    ridges_sted = meijering(img_sted, black_ridges=False)
    
    # Compute SSIM
    ssim_sted.append(structural_similarity(img_sted, ridges_sted))
    
    imsave(out_path + '/ridges_sted_' + img_identifier + '.png', 
           ridges_sted)
    
# Save results
overall_data = pd.DataFrame({'File ExM, ExSRRF': img_identifier_list,
                             'SSIM ExSRRF': ssim_exsrrf,
                             'SSIM Ex': ssim_ex,
                             'File STED': img_identifier_list_sted,
                             'SSIM STED': ssim_sted})
overall_data.to_csv(out_path + 'compare_exsrrf_exm_sted.csv', index=False) 